In [10]:
%pip install pandas
%pip install mlxtend
%pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# How likely is a user to like a movie based on their previous ratings of other movies?

import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import pymysql

hostname = 'riku.shoshin.uwaterloo.ca'
username = 'username'
password = 'password'
database = 'db356_team31'

# Connect to MySQL database
connection = pymysql.connect(
          host=hostname, user=username, password=password, database=database
      )




# Load the data from the MySQL database into a DataFrame
sql_query = "SELECT userId, movie_id, rating FROM Ratings"
data = pd.read_sql(sql_query, connection)
connection.close()


# Convert ratings to binary values (liked or not liked)
data["liked"] = data["rating"] >= 4.0

# Create a binary user-item matrix using pivot_table
basket = data.pivot_table(
    index="userId", columns="movie_id", values="liked", aggfunc="max", fill_value=False
)

# Apply Apriori algorithm
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)


print(rules[["antecedents", "consequents", "support", "confidence"]])

/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


                    antecedents               consequents   support  \
0                        (2355)                       (1)  0.061103   
1                        (3114)                       (1)  0.090909   
2                          (16)                     (296)  0.073025   
3                          (16)                     (593)  0.064083   
4                          (47)                     (296)  0.166915   
...                         ...                       ...       ...   
38668   (7153, 1210, 260, 1198)  (5952, 4993, 2571, 1196)  0.053651   
38669  (7153, 2571, 1196, 1198)   (5952, 4993, 1210, 260)  0.053651   
38670  (7153, 1210, 2571, 1196)   (5952, 4993, 260, 1198)  0.053651   
38671  (7153, 1210, 2571, 1198)   (5952, 4993, 1196, 260)  0.053651   
38672  (7153, 1210, 1196, 1198)   (5952, 4993, 2571, 260)  0.053651   

       confidence  
0        0.759259  
1        0.772152  
2        0.803279  
3        0.704918  
4        0.767123  
...           ...  
38668  